In [1]:
#https://sourceforge.net/projects/swig/files/swigwin/swigwin-4.0.2/swigwin-4.0.2.zip/download?use_mirror=ixpeering

# 1. Import Dependencies

In [2]:
# Import GYM Stuff
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 

# Import helpers
import numpy as np
import random
import os

# Import stable baselines stuff
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

c:\Users\PeterMeyerDr\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\PeterMeyerDr\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\PeterMeyerDr\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
c:\Users\PeterMeyerDr\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


# 2. Types of Spaces

In [3]:
Discrete(3)

Discrete(3)

In [4]:
#checkout values of discrete space
Discrete(3).sample()

0

In [5]:
Box(0,1,shape=(3,3)).sample()

#Box(0,1,shape=(3,).sample() # if only 3 values

array([[0.60862905, 0.81339496, 0.24501622],
       [0.43247885, 0.5254837 , 0.65546   ],
       [0.41309845, 0.0501344 , 0.4302333 ]], dtype=float32)

In [6]:
#Tuple are not supported, but can be generated - does not work

In [7]:
Box(0,255,shape=(3,3), dtype=int).sample()

array([[117, 165, 189],
       [148, 218,  53],
       [ 11,  96,  15]])

In [8]:
Tuple((Discrete(2), Box(0,100, shape=(1,)))).sample()

(0, array([8.770529], dtype=float32))

In [9]:
Dict({'height':Discrete(2), "speed":Box(0,100, shape=(1,))}).sample()

OrderedDict([('height', 1), ('speed', array([30.421865], dtype=float32))])

In [10]:
MultiBinary(4).sample()

array([0, 0, 0, 1], dtype=int8)

In [11]:
MultiDiscrete([5,2,2]).sample()

array([0, 0, 1], dtype=int64)

# 3. Building an Environment

In [12]:
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)  #tap/button up-down-unchanged
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))  #
        # Set start temp
        self.state = 38 + random.randint(-3,3)  #shower start 38 +/- 3
        # Set shower length
        self.shower_length = 60
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 - no change
        # 2 -1 = 1 temperature 
        self.state += action -1 
        # Reduce shower length by 1 second
        self.shower_length -= 1 
        
        # Calculate reward
        if self.state >=37 and self.state <=39: 
            reward =1 
        else: 
            reward = -1 
        
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = np.array([38 + random.randint(-3,3)]).astype(float)
        # Reset shower time
        self.shower_length = 60 
        return self.state

In [13]:
env=ShowerEnv()

c:\Users\PeterMeyerDr\anaconda3\lib\site-packages\gym\spaces\box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [14]:
#check it out - repeat
env.observation_space.sample()

array([53.863777], dtype=float32)

In [15]:
env.observation_space.sample()

array([30.072023], dtype=float32)

In [16]:
env.reset()

array([40.])

In [17]:
from stable_baselines3.common.env_checker import check_env

In [18]:
#check_env(env, warn=True)

# 4. Test Environment

In [19]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:-60
Episode:2 Score:-58
Episode:3 Score:10
Episode:4 Score:-6
Episode:5 Score:-18


In [20]:
env.close()

# 5. Train Model

In [21]:
log_path = os.path.join('Training', 'Logs')

In [22]:
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [23]:
model.learn(total_timesteps=10000)  #play with value

Logging to Training\Logs\PPO_9
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -28.9    |
| time/              |          |
|    fps             | 232      |
|    iterations      | 1        |
|    time_elapsed    | 8        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -29.1       |
| time/                   |             |
|    fps                  | 349         |
|    iterations           | 2           |
|    time_elapsed         | 11          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008663188 |
|    clip_fraction        | 0.0209      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | 0.000548    |

# 6. Save Model

In [24]:
shower_path = os.path.join('Training', 'Saved Models', 'Shower_Model_PPO')

In [25]:
model.save(shower_path)
# model.save('PPO')

In [26]:
# del and restore
del model
model = PPO.load(shower_path, env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [27]:
#evaluate_policy(model, env, n_eval_episodes=10, render=True)
evaluate_policy(model, env, n_eval_episodes=10)

c:\Users\PeterMeyerDr\anaconda3\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(-12.0, 58.787753826796276)